In [ ]:
#import Libraries
import zipfile
from google.colab import drive
import os
import os.path
import cv2
import matplotlib.pyplot as plt
import numpy as np
import keras
from tensorflow.keras.applications import *
from tensorflow.keras import models, layers, optimizers
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

**Summary**

For training the on this small dataset of 100 images in each class, It was obvious that we had to leverage the power of transfer learning

Using pretrained Weights derived by training models on imagenet dataset

I initially decided to use the EfficientNetB0 Network as it had significantly lesser parameters and the highest accuracy compared to other dense networks.But the loss was oscilating similar with Resnet50.

Then i decided to use InceptionV3 with RMSprop and the training loss was going down progressively and good accuracy was obtained.

Dropout and Data Augmentation was done to prevent overfitting



In [ ]:
zip_loc = "/content/drive/My Drive/Xihelm Dataset/dataset cv ai 2021.zip"
DATADIR = "/content/drive/My Drive/Xihelm Dataset/"
drive_loc = "/content/drive/"
CATEGORIES = []
IMG_SIZE = 150
class ImageClassification():

    def __init__(self):
        drive.mount(drive_loc)

    def unzip(self, zip_loc, unzip_loc):
        zip = zipfile.ZipFile(zip_loc, 'r')
        zip.extractall(unzip_loc)
        print("File Extracted")
    
    def labelExtract(self):
        path = DATADIR
        for folder in os.listdir(DATADIR):
            if os.path.isdir(path+folder):
                CATEGORIES.append(folder) 
        return len(CATEGORIES)
    
    def visualizeInit(self, numImage):
        for category in CATEGORIES:
            path = os.path.join(DATADIR,category)
            for img in os.listdir(path)[:numImage]:
                img_arr = cv2.imread(os.path.join(path,img))
                img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
                plt.imshow(img_arr)
                plt.show()

    def data_generation(self):
        image_generator = ImageDataGenerator(rescale=1/255.0, 
                                             validation_split=0.15,
                                             rotation_range = 40,
                                             zoom_range = 0.2,
                                             shear_range = 0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True,
                                             fill_mode="nearest")    

        train_dataset = image_generator.flow_from_directory(batch_size=1,
                                                 directory=DATADIR,
                                                 shuffle=True,
                                                 target_size=(IMG_SIZE, IMG_SIZE), 
                                                 subset="training",
                                                 class_mode='categorical')

        validation_dataset = image_generator.flow_from_directory(batch_size=1,
                                                 directory=DATADIR,
                                                 shuffle=True,
                                                 target_size=(IMG_SIZE, IMG_SIZE), 
                                                 subset="validation",
                                                 class_mode='categorical')
        return train_dataset, validation_dataset

    def saveFeatures(self,featureName, labelName, features, labels):
        
        np.save(drive_loc+DATADIR+featureName, features)
        np.save(drive_loc+DATADIR+labelName, labels)
    
    def loadFeatures(self, featureName, labelName):
        
        features = np.load(drive_loc+DATADIR+featureName)
        labels = np.load(drive_loc+DATADIR+labelName)

        return features, labels

    def create_model(self,NUM_CLASSES):
        base = InceptionV3(include_top=False, weights="imagenet", input_shape=(IMG_SIZE,IMG_SIZE,3))
        for layer in base.layers:
            layer.trainable = False
        model = models.Sequential()
        model.add(base)
        model.add(layers.Flatten())
        model.add(layers.Dense(1024, activation='relu'))
        dropout_rate = 0.2
        model.add(layers.Dropout(dropout_rate, name="dropout_out"))

        model.add(layers.Dense(NUM_CLASSES, activation="softmax", name="fc_out"))
        model.summary()
        return model

    def fitModel(self, model,train,valid, epochs):
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
        model.compile(loss = "categorical_crossentropy",
                      optimizer=optimizer,
                      metrics=["accuracy"])
        history = model.fit_generator(train, epochs=epochs, validation_data=valid)
        return history
    
    def dataPrep(self, filepath):
        img = cv2.imread(filepath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_array = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)
        
        return img_array
ic = ImageClassification()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# ic.unzip(zip_loc,DATADIR)
NUM_CLASSES = ic.labelExtract()
print(CATEGORIES, NUM_CLASSES)

['animals', 'buildings', 'landscapes'] 3


In [ ]:
ic.visualizeInit(5)

In [ ]:
train_dataset, validation_dataset = ic.data_generation()

Found 255 images belonging to 3 classes.
Found 45 images belonging to 3 classes.


In [ ]:
model = ic.create_model(NUM_CLASSES)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 3, 3, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
dropout_out (Dropout)        (None, 1024)              0         
_________________________________________________________________
fc_out (Dense)               (None, 3)                 3075      
Total params: 40,681,251
Trainable params: 18,878,467
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
history = ic.fitModel(model, train_dataset, validation_dataset, 60)

Epoch 1/60


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


255/255 [==============================] - 11s 31ms/step - loss: 25.6606 - accuracy: 0.5765 - val_loss: 26.1423 - val_accuracy: 0.4889
Epoch 2/60
255/255 [==============================] - 7s 26ms/step - loss: 10.9223 - accuracy: 0.6431 - val_loss: 3.0801 - val_accuracy: 0.8444
Epoch 3/60
255/255 [==============================] - 7s 26ms/step - loss: 8.8894 - accuracy: 0.6902 - val_loss: 27.9907 - val_accuracy: 0.4222
Epoch 4/60
255/255 [==============================] - 7s 26ms/step - loss: 7.3565 - accuracy: 0.7137 - val_loss: 1.9182 - val_accuracy: 0.8222
Epoch 5/60
255/255 [==============================] - 7s 26ms/step - loss: 4.1605 - accuracy: 0.7765 - val_loss: 2.6539 - val_accuracy: 0.8222
Epoch 6/60
255/255 [==============================] - 7s 26ms/step - loss: 4.9540 - accuracy: 0.7216 - val_loss: 2.8754 - val_accuracy: 0.7556
Epoch 7/60
255/255 [==============================] - 7s 26ms/step - loss: 3.0105 - accuracy: 0.8039 - val_loss: 2.4406 - val_accuracy: 0.7556
Epoch

Model Training Accuracy - 92.04%, Model Validaiton Set Accuracy - 84.4%

In [ ]:
model.save(DATADIR+"incepv3.model")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Xihelm Dataset/incepv3.model/assets
